In [7]:
import cv2
import json
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# MMPOSE 관련 라이브러리
from mmpose.apis import init_model as init_pose_estimator, inference_topdown
from mmpose.structures import merge_data_samples, split_instances

# ====================================================
# 1. 설정 및 경로 정의
# ====================================================
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_sam3/")
CSV_PATH = DATA_DIR / "metadata.csv"

# Sapiens Config & Checkpoint
POSE_CFG = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_sapiens_labeling/configs/sapiens/sapiens_0.3b-210e_coco-1024x768.py"
POSE_CKPT = DATA_DIR / "checkpoints/sapiens/pose/sapiens_0.3b_coco_best_coco_AP_796.pth"

# CSV 불러오기
df = pd.read_csv(CSV_PATH)

target = 0
COMMON_PATH = df.loc[target, "common_path"]
VIDEO_PTH = df.loc[target, "video_path"]

# 경로 설정
FRAME_DIR = DATA_DIR / "1_FRAME" / COMMON_PATH
SAVE_JSON_DIR = DATA_DIR / "samplussapiens" / COMMON_PATH
BBOX_PATH = DATA_DIR / "test" / COMMON_PATH / "video_segments.json"

print(f"[INFO] Target: {COMMON_PATH}")
print(f"[INFO] BBox Path: {BBOX_PATH}")
print(f"[INFO] Save JSON Dir: {SAVE_JSON_DIR}")

# ====================================================
# 2. 유틸리티 함수 (✅ 재귀적 변환 기능 추가 - 수정됨)
# ====================================================
def to_py(obj):
    """
    복잡한 중첩 구조(dict, list) 내의 모든 NumPy 타입을 
    JSON 직렬화 가능한 Python 기본 타입으로 재귀 변환합니다.
    """
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.float32, np.float64, np.floating)):
        return float(obj)
    if isinstance(obj, (np.int32, np.int64, np.integer)):
        return int(obj)
    if isinstance(obj, dict):
        return {k: to_py(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_py(v) for v in obj]
    return obj

# ====================================================
# 3. Sapiens 추출 함수
# ====================================================
def run_sapiens_with_scaled_bbox(
    frame_dir, json_out_dir, bbox_path, pose_cfg, pose_ckpt, 
    device="cuda:0", 
    orig_w=1920, orig_h=1080
):
    frame_dir, json_out_dir = Path(frame_dir), Path(json_out_dir)
    bbox_path = Path(bbox_path)

    # 1. BBox JSON 로드
    if not bbox_path.exists():
        print(f"[ERROR] BBox 파일 없음: {bbox_path}")
        return
    with open(bbox_path, "r", encoding="utf-8") as f:
        bbox_data = json.load(f)

    # 2. 결과 폴더 초기화
    if json_out_dir.exists():
        shutil.rmtree(json_out_dir)
    json_out_dir.mkdir(parents=True, exist_ok=True)

    # 3. 현재 프레임 해상도 확인 및 스케일 계산
    frame_files = sorted(list(frame_dir.glob("*.jpg")))
    if not frame_files:
        print("[ERROR] 프레임 이미지가 없습니다.")
        return

    sample_img = cv2.imread(str(frame_files[0]))
    curr_h, curr_w = sample_img.shape[:2]

    scale_x = curr_w / orig_w
    scale_y = curr_h / orig_h
    
    print(f"[INFO] 해상도 변환: {orig_w}x{orig_h} -> {curr_w}x{curr_h}")
    print(f"[INFO] Scale Factor: X={scale_x:.4f}, Y={scale_y:.4f}")

    # 4. 모델 로드
    pose_estimator = init_pose_estimator(
        str(pose_cfg), str(pose_ckpt), device=device,
        cfg_options=dict(model=dict(test_cfg=dict(output_heatmaps=False)))
    )

    # 5. 추론 루프 (Sequential)
    saved_count = 0
    
    for fpath in tqdm(frame_files, desc="Sapiens Processing"):
        frame_idx_str = str(int(fpath.stem)) # "000123" -> "123"

        # 1) BBox 정보 확인
        if frame_idx_str not in bbox_data:
            continue
        
        instances = bbox_data[frame_idx_str]
        if not instances:
            continue

        # 2) BBox 스케일링 및 수집
        frame_boxes = []
        for _, val in instances.items():
            if "bbox" in val:
                box = np.array(val["bbox"], dtype=np.float32)
                box[0] *= scale_x
                box[1] *= scale_y
                box[2] *= scale_x
                box[3] *= scale_y
                frame_boxes.append(box)

        if not frame_boxes:
            continue
        
        # numpy 변환 (N, 4)
        bbox_np = np.array(frame_boxes)

        try:
            # 단일 프레임 추론
            results = inference_topdown(pose_estimator, str(fpath), bbox_np)
        except Exception as e:
            print(f"[WARN] Inference failed for {fpath.name}: {e}")
            continue

        # 3) 결과 저장
        for res in results:
            data_sample = merge_data_samples([res])
            inst = data_sample.get("pred_instances", None)
            
            if inst is None: continue
            inst_list = split_instances(inst)

            payload = {
                "frame_index": int(fpath.stem),
                "meta_info": pose_estimator.dataset_meta, # 여기에 numpy가 섞여있음
                "instance_info": inst_list,               # 여기에도 numpy가 섞여있음
                "source": "scaled_bbox_json"
            }
            
            out_path = json_out_dir / f"{fpath.stem}.json"
            
            # ✅ to_py가 이제 재귀적으로 돌면서 모든 numpy를 제거함
            with open(out_path, "w", encoding="utf-8") as f:
                json.dump(to_py(payload), f, ensure_ascii=False, indent=2)
            
            saved_count += 1

    print(f"[✅] Keypoint 추출 완료: {saved_count} frames saved in {json_out_dir}")

# ====================================================
# 4. 실행
# ====================================================
if __name__ == "__main__":
    run_sapiens_with_scaled_bbox(
        frame_dir=FRAME_DIR,
        json_out_dir=SAVE_JSON_DIR,
        bbox_path=BBOX_PATH,
        pose_cfg=POSE_CFG,
        pose_ckpt=POSE_CKPT,
        orig_w=1920, 
        orig_h=1080
    )
    
    print(f"\n[🏁 완료] JSON 저장 위치: {SAVE_JSON_DIR}")

[INFO] Target: AI_dataset/N01/N01_Treatment/diagonal__biceps_curl
[INFO] BBox Path: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/test/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl/video_segments.json
[INFO] Save JSON Dir: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/samplussapiens/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl
[INFO] 해상도 변환: 1920x1080 -> 1280x720
[INFO] Scale Factor: X=0.6667, Y=0.6667
Loads checkpoint by local backend from path: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/checkpoints/sapiens/pose/sapiens_0.3b_coco_best_coco_AP_796.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: head.deconv_layers.1.weight, head.deconv_layers.1.bias, head.deconv_layers.1.running_mean, head.deconv_layers.1.running_var, head.deconv_layers.4.weight, head.deconv_layers.4.bias, head.deconv_layers.4.running_mean, head.deconv_layers.4.running_var, head.conv_layers.1.weight, head.

Sapiens Processing:  18% 430/2432 [03:43<17:24,  1.92it/s]/opt/conda/lib/python3.10/site-packages/mmpose/structures/bbox/transforms.py:357: RuntimeWarning: divide by zero encountered in divide
  scale_x = (output_size[0] - 1) / scale[0]
/opt/conda/lib/python3.10/site-packages/mmpose/structures/bbox/transforms.py:358: RuntimeWarning: divide by zero encountered in divide
  scale_y = (output_size[1] - 1) / scale[1]
/opt/conda/lib/python3.10/site-packages/mmpose/structures/bbox/transforms.py:360: RuntimeWarning: invalid value encountered in scalar multiply
  warp_mat[0, 1] = -math.sin(rot_rad) * scale_x
/opt/conda/lib/python3.10/site-packages/mmpose/structures/bbox/transforms.py:361: RuntimeWarning: invalid value encountered in scalar multiply
  warp_mat[0, 2] = scale_x * (-0.5 * input_size[0] * math.cos(rot_rad) +
/opt/conda/lib/python3.10/site-packages/mmpose/structures/bbox/transforms.py:364: RuntimeWarning: invalid value encountered in scalar multiply
  warp_mat[1, 0] = math.sin(rot_ra

[✅] Keypoint 추출 완료: 2432 frames saved in /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/samplussapiens/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl

[🏁 완료] JSON 저장 위치: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/samplussapiens/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl


In [ ]:
kpt_path = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/samplussapiens/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl"

In [3]:
import cv2
import json
import numpy as np
from pathlib import Path
from tqdm import tqdm
import pandas as pd

# ====================================================
# 1. Settings & Paths
# ====================================================
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
CSV_PATH = DATA_DIR / "metadata.csv"

# Load Metadata
df = pd.read_csv(CSV_PATH)
target = 0
COMMON_PATH = df.loc[target, "common_path"]

# Define Paths
FRAME_DIR = DATA_DIR / "1_FRAME" / COMMON_PATH
KPT_DIR = DATA_DIR / "samplussapiens" / COMMON_PATH
OUTPUT_VIDEO_PATH = DATA_DIR / "test" / f"{COMMON_PATH}_overlay.mp4"

# Visualization Settings
BBOX_COLOR = (0, 255, 0)      # Green
BBOX_THICKNESS = 2
SKELETON_THICKNESS = 2
KEYPOINT_RADIUS = 4
FONT = cv2.FONT_HERSHEY_SIMPLEX

# ✅ 중요: 점수가 낮아도 보이게 하기 위해 0.0으로 설정 (기존 0.3 -> 0.0)
CONF_THRESHOLD = 0.0 

print(f"[INFO] Frames: {FRAME_DIR}")
print(f"[INFO] JSONs:  {KPT_DIR}")
print(f"[INFO] Output: {OUTPUT_VIDEO_PATH}")

# ====================================================
# 2. Rendering Function
# ====================================================
def render_video(frame_dir, json_dir, output_path, fps=30):
    frame_dir = Path(frame_dir)
    json_dir = Path(json_dir)
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    frame_files = sorted(list(frame_dir.glob("*.jpg")))
    if not frame_files:
        print("[ERROR] No frames found.")
        return

    first_frame = cv2.imread(str(frame_files[0]))
    height, width, _ = first_frame.shape
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    print(f"[INFO] Starting rendering... ({width}x{height} @ {fps}fps)")

    for frame_path in tqdm(frame_files, desc="Rendering Video"):
        frame = cv2.imread(str(frame_path))
        if frame is None: continue
            
        frame_idx_str = frame_path.stem
        json_path = json_dir / f"{frame_idx_str}.json"

        if json_path.exists():
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            
            meta = data.get("meta_info", {})
            skeleton_links = meta.get("skeleton_links", [])
            link_colors = meta.get("skeleton_link_colors", [])
            kpt_colors = meta.get("keypoint_colors", [])

            instances = data.get("instance_info", [])
            for inst in instances:
                # 1. Draw Bounding Box
                if "bbox" in inst:
                    bbox = inst["bbox"]
                    if isinstance(bbox[0], list): 
                        bbox = bbox[0]
                    x1, y1, x2, y2 = map(int, bbox)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), BBOX_COLOR, BBOX_THICKNESS)

                # 2. Draw Skeleton & Keypoints
                if "keypoints" in inst:
                    kpts = np.array(inst["keypoints"])
                    scores = inst.get("keypoint_scores", np.ones(len(kpts)))

                    # Draw Links
                    for idx, link in enumerate(skeleton_links):
                        i, j = link
                        if i >= len(kpts) or j >= len(kpts): continue
                        
                        # ✅ 수정됨: CONF_THRESHOLD 변수 사용 (0.0 이상이면 그리기)
                        if scores[i] < CONF_THRESHOLD or scores[j] < CONF_THRESHOLD: continue

                        pt1 = tuple(map(int, kpts[i]))
                        pt2 = tuple(map(int, kpts[j]))
                        
                        color = tuple(link_colors[idx]) if idx < len(link_colors) else (255, 0, 0)
                        color = color[::-1] 
                        cv2.line(frame, pt1, pt2, color, SKELETON_THICKNESS)

                    # Draw Points
                    for i, (x, y) in enumerate(kpts):
                        # ✅ 수정됨: CONF_THRESHOLD 변수 사용
                        if scores[i] < CONF_THRESHOLD: continue
                        
                        color = tuple(kpt_colors[i]) if i < len(kpt_colors) else (0, 0, 255)
                        color = color[::-1]
                        cv2.circle(frame, (int(x), int(y)), KEYPOINT_RADIUS, color, -1)

        out.write(frame)

    out.release()
    print(f"[DONE] Video saved to: {output_path}")

if __name__ == "__main__":
    render_video(FRAME_DIR, KPT_DIR, OUTPUT_VIDEO_PATH)

[INFO] Frames: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/1_FRAME/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl
[INFO] JSONs:  /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/samplussapiens/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl
[INFO] Output: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/test/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl_overlay.mp4
[INFO] Starting rendering... (1280x720 @ 30fps)


Rendering Video: 100% 2432/2432 [00:27<00:00, 89.55it/s] 


[DONE] Video saved to: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/test/AI_dataset/N01/N01_Treatment/diagonal__biceps_curl_overlay.mp4


In [5]:
import cv2
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# ====================================================
# 1. Settings & Paths
# ====================================================
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
CSV_PATH = DATA_DIR / "metadata.csv"

# Load Metadata
df = pd.read_csv(CSV_PATH)
target = 1
VIDEO_PTH = Path(df.loc[target, "video_path"])

# 저장할 폴더 설정
OUTPUT_VIDEO_DIR = DATA_DIR / "test" / "frame_change"
OUTPUT_VIDEO_DIR.mkdir(parents=True, exist_ok=True)

print(f"🚀 영상 변환 시작: {VIDEO_PTH.name}")
print(f"📂 저장 경로: {OUTPUT_VIDEO_DIR}")

# ====================================================
# 2. Resizing Logic
# ====================================================
def resize_video_multi_output(input_path, output_dir):
    # 1. 비디오 열기
    cap = cv2.VideoCapture(str(input_path))
    if not cap.isOpened():
        print("❌ 영상을 열 수 없습니다.")
        return

    # 원본 정보 읽기
    org_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    org_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"ℹ️ 원본 정보: {org_width}x{org_height} @ {fps}fps")

    # 2. 목표 해상도 4개 정의 (너비, 높이) - 16:9 비율 유지
    # SAM 등에 넣기 좋은 사이즈들로 구성했습니다.
    target_resolutions = [
        (1280, 720),  # HD (720p)
        (1024, 576),  # SAM 친화적 너비 (1024px)
        (640, 360),   # nHD (360p) - 속도 매우 빠름
        (480, 270)    # 저해상도 - 테스트용
    ]

    # 3. Writer 객체들 초기화
    writers = []
    
    for w, h in target_resolutions:
        # 파일명 자동 생성 (예: video_1280x720.mp4)
        out_name = output_dir / f"{input_path.stem}_{w}x{h}.mp4"
        
        # MP4 코덱 설정
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        writer = cv2.VideoWriter(str(out_name), fourcc, fps, (w, h))
        
        if not writer.isOpened():
            print(f"❌ Writer 초기화 실패: {out_name}")
            return
            
        writers.append(writer)
        print(f"   - 준비됨: {out_name.name}")

    # 4. 프레임 처리 루프 (한 번 읽어서 네 번 쓰기)
    print("-" * 60)
    pbar = tqdm(total=total_frames, desc="Processing")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # 각 해상도별로 리사이즈 후 저장
        for idx, (w, h) in enumerate(target_resolutions):
            # 리사이즈 (AREA 보간법이 축소 시 화질이 가장 좋음)
            resized_frame = cv2.resize(frame, (w, h), interpolation=cv2.INTER_AREA)
            writers[idx].write(resized_frame)
            
        pbar.update(1)

    # 5. 자원 해제
    cap.release()
    for writer in writers:
        writer.release()
    pbar.close()
    
    print("-" * 60)
    print("✅ 모든 변환이 완료되었습니다!")

# 실행
resize_video_multi_output(VIDEO_PTH, OUTPUT_VIDEO_DIR)

🚀 영상 변환 시작: diagonal__hip_extension.mp4
📂 저장 경로: /workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data/test/frame_change
ℹ️ 원본 정보: 1920x1080 @ 29.97002997002997fps
   - 준비됨: diagonal__hip_extension_1280x720.mp4
   - 준비됨: diagonal__hip_extension_1024x576.mp4
   - 준비됨: diagonal__hip_extension_640x360.mp4
   - 준비됨: diagonal__hip_extension_480x270.mp4
------------------------------------------------------------


Processing: 100% 320/320 [00:11<00:00, 27.60it/s]

------------------------------------------------------------
✅ 모든 변환이 완료되었습니다!
